## EDA


In [1]:
import pandas as pd
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import seaborn as sns
import h2o
from h2o.automl import H2OAutoML
import numpy as np

import pandas as pd
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

from imblearn.under_sampling import RandomUnderSampler
import pandas as pd

In [2]:


# Read the CSV file into a DataFrame
df = pd.read_csv("merged4.csv")

# Display the DataFrame
df

,Age,gender,ethnicity,insurance,id,label,222042,225158,225943,229234,...,O71,O48,O11,O45,L22,Z28,M87,B33,S69,D02
0,46,F,WHITE,Other,31205490,0,0.0,0.000000,0.000000,0.0,...,0,0,0,0,0,0,0,0,0,0
1,46,F,WHITE,Other,31205490,0,0.0,0.000000,0.000000,0.0,...,0,0,0,0,0,0,0,0,0,0
2,46,F,WHITE,Other,31205490,0,0.0,114.666670,0.000000,0.0,...,0,0,0,0,0,0,0,0,0,0
3,46,F,WHITE,Other,31205490,0,0.0,114.666670,0.000000,0.0,...,0,0,0,0,0,0,0,0,0,0
4,46,F,WHITE,Other,31205490,0,0.0,114.666670,0.000000,0.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80455,47,M,UNKNOWN,Other,32880062,0,0.0,99.099998,11.312849,0.0,...,0,0,0,0,0,0,0,0,0,0
80456,47,M,UNKNOWN,Other,32880062,0,0.0,99.099998,88.687214,0.0,...,0,0,0,0,0,0,0,0,0,0
80457,47,M,UNKNOWN,Other,32880062,0,0.0,99.099998,88.687214,0.0,...,0,0,0,0,0,0,0,0,0,0
80458,47,M,UNKNOWN,Other,32880062,0,0.0,99.099998,88.687214,0.0,...,0,0,0,0,0,0,0,0,0,0


# DATA CLEANING

### REMOVING features with 80% missing data

#### Looking for NANS

In [3]:
# Calculate missing percentage for each column
for column in df.columns:
    missing_count = df[column].isnull().sum()
    total_count = len(df[column])
    missing_percentage = (missing_count / total_count) * 100
    print(f"Missing percentage in {column}: {missing_percentage:.2f}%")



Missing percentage in Age: 0.00%
Missing percentage in gender: 0.00%
Missing percentage in ethnicity: 0.00%
Missing percentage in insurance: 0.00%
Missing percentage in id: 0.00%
Missing percentage in label: 0.00%
Missing percentage in 222042: 0.00%
Missing percentage in 225158: 0.00%
Missing percentage in 225943: 0.00%
Missing percentage in 229234: 0.00%
Missing percentage in 220949: 0.00%
Missing percentage in 221289: 0.00%
Missing percentage in 221668: 0.00%
Missing percentage in 221906: 0.00%
Missing percentage in 222315: 0.00%
Missing percentage in 225168: 0.00%
Missing percentage in 225942: 0.00%
Missing percentage in 225944: 0.00%
Missing percentage in 226089: 0.00%
Missing percentage in 227525: 0.00%
Missing percentage in 227529: 0.00%
Missing percentage in 227536: 0.00%
Missing percentage in 220864: 0.00%
Missing percentage in 221749: 0.00%
Missing percentage in 222056: 0.00%
Missing percentage in 225151: 0.00%
Missing percentage in 225152: 0.00%
Missing percentage in 225823: 

#### dropping column if 80% is the same

In [4]:
# Calculate the similarity percentage for each column
similarity_threshold = 0.8  # 80% threshold

X2 = df.drop('label', axis=1)  # Features
y2 = df['label']  # Target variable

columns_to_drop = []
for column in X2.columns:
    similarity = X2[column].value_counts(normalize=True).max()
    if similarity >= similarity_threshold:
        columns_to_drop.append(column)

# Drop columns with high similarity
X2 = X2.drop(columns=columns_to_drop, axis=1)

print("DataFrame after dropping columns with high similarity:")
print(X2)

DataFrame after dropping columns with high similarity:
       Age gender ethnicity insurance        id      225158     225943  \
0       46      F     WHITE     Other  31205490    0.000000   0.000000   
1       46      F     WHITE     Other  31205490    0.000000   0.000000   
2       46      F     WHITE     Other  31205490  114.666670   0.000000   
3       46      F     WHITE     Other  31205490  114.666670   0.000000   
4       46      F     WHITE     Other  31205490  114.666670   0.000000   
...    ...    ...       ...       ...       ...         ...        ...   
80455   47      M   UNKNOWN     Other  32880062   99.099998  11.312849   
80456   47      M   UNKNOWN     Other  32880062   99.099998  88.687214   
80457   47      M   UNKNOWN     Other  32880062   99.099998  88.687214   
80458   47      M   UNKNOWN     Other  32880062   99.099998  88.687214   
80459   47      M   UNKNOWN     Other  32880062   99.099998  88.687214   

       220949  226559  220224  ...  Z95  E78  I47  N17  

## XGBOOST with dropped values contains ~ 100 or os features

In [5]:



# Convert categorical columns into dummy/one-hot encoding
X2 = pd.get_dummies(X2)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X2, y2, test_size=0.2, random_state=42)

print("Shapes after train-test split:")
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)

# Initialize an XGBoost classifier
xgb_classifier80 = xgb.XGBClassifier(objective='binary:logistic', eval_metric='logloss')

# Fit the classifier to the training data
xgb_classifier80.fit(X_train, y_train)

# Make predictions on the test set
y_pred = xgb_classifier80.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

print("Accuracy: {:.2f}%".format(accuracy * 100))
print("Confusion Matrix:\n", conf_matrix)
print("Classification Report:\n", class_report)

Shapes after train-test split:
X_train shape: (64368, 147)
y_train shape: (64368,)
X_test shape: (16092, 147)
y_test shape: (16092,)
Accuracy: 99.63%
Confusion Matrix:
 [[13943     2]
 [   58  2089]]
Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00     13945
           1       1.00      0.97      0.99      2147

    accuracy                           1.00     16092
   macro avg       1.00      0.99      0.99     16092
weighted avg       1.00      1.00      1.00     16092



In [6]:
# # Calculate the percentage of missing values in each column
# missing_percentage = df.isnull().mean() * 100
# print(missing_percentage)
# # Define the threshold for dropping columns (80% missing values)
# threshold = 80

# # Filter columns where missing values exceed the threshold
# columns_to_drop = missing_percentage[missing_percentage > threshold].index

# # Drop the columns from the DataFrame
# df = df.drop(columns=columns_to_drop)
# df = df.drop('id', axis=1)  # X contains the features
# print("DataFrame after dropping columns:")
# print(df)

In [7]:
# # Separate features (X) and target variable (y)
# x = df.drop('label', axis=1)  # Contains feature variables (all columns except 'label')
# y = df['label']  # Contains the target variable ('label' column)

data=df

## Simple XGboost on full dataset

In [8]:


# Assuming 'data' is your pandas DataFrame that includes both features and the target variable 'label'
# Separate features and target variable
X = data.drop('label', axis=1)  # Features
y = data['label']  # Target variable

# Convert categorical columns into dummy/one-hot encoding
X = pd.get_dummies(X)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Shapes after train-test split:")
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)

# Initialize an XGBoost classifier
xgb_classifier = xgb.XGBClassifier(objective='binary:logistic', eval_metric='logloss')

# Fit the classifier to the training data
xgb_classifier.fit(X_train, y_train)

# Make predictions on the test set
y_pred = xgb_classifier.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

print("Accuracy: {:.2f}%".format(accuracy * 100))
print("Confusion Matrix:\n", conf_matrix)
print("Classification Report:\n", class_report)

Shapes after train-test split:
X_train shape: (64368, 1874)
y_train shape: (64368,)
X_test shape: (16092, 1874)
y_test shape: (16092,)
Accuracy: 99.40%
Confusion Matrix:
 [[13943     2]
 [   94  2053]]
Classification Report:
               precision    recall  f1-score   support

           0       0.99      1.00      1.00     13945
           1       1.00      0.96      0.98      2147

    accuracy                           0.99     16092
   macro avg       1.00      0.98      0.99     16092
weighted avg       0.99      0.99      0.99     16092



In [9]:
from piml import Experiment
exp = Experiment()

#### As we can see here the dataset is now clean and prediction with this model is good now we will go into l1 regularization 

# testing L1 regularization for feature reduction

In [10]:
# from sklearn.linear_model import Lasso
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import StandardScaler
# from sklearn.datasets import make_regression

# # Assuming 'data' is your pandas DataFrame that includes both features and the target variable 'label'
# # Separate features and target variable
# X = data.drop('label', axis=1)  # Features
# y = data['label']  # Target variable

# # Split the data into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# # Standardize features (important for regularization techniques)
# scaler = StandardScaler()
# X_train = scaler.fit_transform(X_train)
# X_test = scaler.transform(X_test)

# # Apply L1 regularization using Lasso
# lasso = Lasso(alpha=0.1)  # Alpha is the regularization strength (hyperparameter)
# lasso.fit(X_train, y_train)

# # Evaluate the model
# train_score = lasso.score(X_train, y_train)
# test_score = lasso.score(X_test, y_test)

# print(f"Training R^2 score: {train_score}")
# print(f"Testing R^2 score: {test_score}")

# # Get the selected features and coefficients
# selected_features = [index for index, coef in enumerate(lasso.coef_) if coef != 0]
# selected_coefficients = lasso.coef_[lasso.coef_ != 0]

# print("Selected features:", selected_features)
# print("Corresponding coefficients:", selected_coefficients)


HTML(value='\n        <style>\n\n        .left-label {\n            width: 30%;\n        }\n\n        .card-pa…

#### not needed found results was negligable when comparing to just putting the whole dataset into an xgboost

## RFE with xgboost to reduce features

In [11]:
# from xgboost import XGBClassifier
# from sklearn.feature_selection import RFE
# from sklearn.datasets import make_classification

# # Generating some example data
# X, y = make_classification(n_samples=1000, n_features=300, random_state=42)

# # Initialize the XGBoost classifier
# xgb_model = XGBClassifier()

# # Initialize RFE with the XGBoost model and specify the number of features you want to retain
# rfe = RFE(estimator=xgb_model, n_features_to_select=10)

# # Fit RFE
# rfe.fit(X, y)

# # Selected features
# selected_features = rfe.support_

# print("Selected features:", selected_features)

HTML(value='\n        <style>\n\n        .left-label {\n            width: 30%;\n        }\n\n        .card-pa…

#### not needed found results was negligable when comparing to just putting the whole dataset into an xgboost

## downsampling

In [17]:
df

HTML(value='\n        <style>\n\n        .left-label {\n            width: 30%;\n        }\n\n        .card-pa…

,Age,gender,ethnicity,insurance,id,label,222042,225158,225943,229234,220949,221289,221668,221906,222315,225168,225942,225944,226089,227525,227529,227536,220864,221749,222056,225151,225152,225823,229624,222168,225828,225159,228315,221662,229597,225170,225825,228339,229630,223258,225945,225153,225910,225155,227522,227523,225157,225147,221744,220970,...,C88,F93,O40,F09,J67,J21,A07,O22,X74,Z65,F72,S29,Z09,S53,B50,C94,G65,X10,A88,Z47,M30,O67,V28,E40,F63,A77,I78,Q65,R52,M08,R77,M05,V94,L92,S85,C46,Q79,M04,V27,M53,O71,O48,O11,O45,L22,Z28,M87,B33,S69,D02
0,46,F,WHITE,Other,31205490,0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,46,F,WHITE,Other,31205490,0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,46,F,WHITE,Other,31205490,0,0.0,114.666670,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,50.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,46,F,WHITE,Other,31205490,0,0.0,114.666670,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,49.999999,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,46,F,WHITE,Other,31205490,0,0.0,114.666670,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,49.999999,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80455,47,M,UNKNOWN,Other,32880062,0,0.0,99.099998,11.312849,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.542,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,113.128488,999.999962,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
80456,47,M,UNKNOWN,Other,32880062,0,0.0,99.099998,88.687214,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.542,0.000000,249.99999,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,886.872201,999.999997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
80457,47,M,UNKNOWN,Other,32880062,0,0.0,99.099998,88.687214,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.542,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,886.872201,999.999997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
80458,47,M,UNKNOWN,Other,32880062,0,0.0,99.099998,88.687214,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.542,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,886.872201,1000.000043,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,

In [ ]:
import pandas as pd
from imblearn.under_sampling import RandomUnderSampler

# Assuming 'df' is your DataFrame that you want to undersample

# Create an instance of RandomUnderSampler
rus = RandomUnderSampler(random_state=42)  # You can set the random_state for reproducibility

# Assuming the column 'target' represents the target variable that needs undersampling
X = df.drop('target', axis=1)  # Features
y = df['target']  # Target variable

# Undersampling the dataset
X_resampled, y_resampled = rus.fit_resample(X, y)

# Combine the features and target variable into a new DataFrame
resampled_df = pd.concat([X_resampled, y_resampled], axis=1)

# 'resampled_df' now contains the undersampled DataFrame


In [14]:
print(len(Xds))

HTML(value='\n        <style>\n\n        .left-label {\n            width: 30%;\n        }\n\n        .card-pa…

80460


## AUTOML

In [15]:
# # Start the H2O cluster
# h2o.init()



# # Convert the pandas DataFrame to an H2OFrame
# h2o_df = h2o.H2OFrame(df)

# # Separate features (X) and target variable (y)
# x = df.drop('label', axis=1)  # Contains feature variables (all columns except 'label')
# y = df['label']  # Contains the target variable ('label' column)


# # Run H2O AutoML
# aml = H2OAutoML(max_models=10, seed=1)
# aml.train(y='label', training_frame=h2o_df)

# # View the leaderboard
# lb = aml.leaderboard
# print(lb)


HTML(value='\n        <style>\n\n        .left-label {\n            width: 30%;\n        }\n\n        .card-pa…

In [16]:
# Select the top model
top_model = aml.leader

# Retrieve details of the top model
model_details = h2o.get_model(top_model.model_id)

# View the top model details
print(model_details)

HTML(value='\n        <style>\n\n        .left-label {\n            width: 30%;\n        }\n\n        .card-pa…

NameError: name 'aml' is not defined

In [ ]:
best_model = top_model

# For stacked ensemble, obtain variable importance of the base models
if best_model.algo == "stackedensemble":
    base_models = best_model.varimp(True)
    
    # Plot the variable importance for the base models
    for model in base_models['base']:
        base_model = h2o.get_model(model)
        if base_model is not None:
            base_model.varimp_plot()


TypeError: 'NoneType' object is not subscriptable